<a href="https://colab.research.google.com/github/shin0105/4YP/blob/master/error.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#installing packages
!pip install jaxopt
!pip install tinygp

#importing packages
import numpy as np
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
import jaxopt
from tinygp import GaussianProcess, kernels, transforms
import scipy
jax.config.update("jax_enable_x64", True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 5.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 769 kB/s 


In [2]:
#DROP-WAVE FUNCTION using tinygp

#DROP-WAVE FUNCTION description
X = np.linspace((-5.12,-5.12),(5.12,5.12),10)
yerr = 0.01
y = -(1+np.cos(12*np.sqrt(X[:, 0]**2+X[:, 1]**2)))/(0.5*(X[:, 0]**2+X[:, 1]**2)+2) + yerr * np.random.normal(
    size=len(X))


X1,X2 = np.meshgrid(X[:,0], X[:,1])
y_true = -(1+np.cos(12*np.sqrt(X1**2+X2**2)))/(0.5*(X1**2+X2**2)+2)
X_pred = np.vstack((X1.flatten(), X2.flatten())).T



#Training data
def train_gp(nparams, build_gp_func):
    @jax.jit
    def loss(params):
        return -build_gp_func(params).log_probability(y)

    params = {
        "log_amp": np.float64(0.0),
        "log_scale": np.zeros(nparams),
    }
    solver = jaxopt.ScipyMinimize(fun=loss)
    soln = solver.run(params)
    return build_gp_func(soln.params)

#Kernel = Matern v=5/2, uncorrelated 
def build_gp_uncorr(params):
    kernel = jnp.exp(params["log_amp"]) * transforms.Linear(
        jnp.exp(-params["log_scale"]), kernels.Matern52()
    )
    return GaussianProcess(kernel, X, diag=yerr**2)

uncorr_gp = train_gp(2, build_gp_uncorr)

y_pred = uncorr_gp.condition(y, X_pred).gp.loc.reshape(y_true.shape)

#printing results, comparing trained(predicted) data vs. true value
print(f"Predicted minimum y value using GP: {y_pred.min()}")

print(f"Minimum y value from sampled points: {y_true.min()}")

AttributeError: ignored